# Predict Survival on Titanic  

In this notebook, I will fit and try to predict survival in the Titanic using the Titanic dataset which is available on [kaggle](https://www.kaggle.com/c/titanic/data).

In [1]:
# Import packages
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
## 